# Top Trendings do Twitter
**Objetivos:** Desenvolver uma aplicação para monitorar o top trending e seu devido volume na rede social com o passar do tempo.

## Funcionalidades
**Requisitos funcionais obrigatorios**:
- [x] Utilizar a api do twitter para consumir o top trending e seu devido volume.
- [x] Armazenar o histórico das trendings em um banco de dados noSQL.
- [ ] Utilizar alguma biblioteca gráfica para exibir os dados.
**Requisito funcional opcional**:
- [ ] Utilizar IA para análise de sentimento dos tweets referentes aos top trendings

### Instação das Bibliotecas
Para instalar as bibliotecas, será usado um arquivo de texto com todas as dependências necessarias para este projeto funcionar.

In [1]:
! python -m pip install -r requirements.txt

### Imports

In [2]:
import json
import pprint

import requests

from modules import mongoDb
from modules import fileToDic as fd
from modules import apiKeys as api
from modules import mongoEnv

## Funcionalidade 1: Consome o top trending e seu devido volume.

1. Abrir o arquivo `WOEID.txt` para poder ler os paises e seu identificadores.
2. Usando a função `dictionary` que esta no arquivo `fileToDic`, o arquivo com os paises e seus respectivos valores serão convertidos em dicionario.
3. Usando o dicionario `key` que está no arquivo `apiKeys`, o valor do `BEARER_TOKEN` no arquivo `.env` será atribuido a varivel `bearer_token`. O Bearer Token será usado na autenticação da url que retorna os *trends* de um determinado pais.
4. Para buscar o trending de um determinado pais iremos usar a url: https://api.twitter.com/1.1/trends/place.json
5. Entretanto, devemos passar o parametro que no caso é o id de um pais.

Fonte: https://developer.twitter.com/en/docs/twitter-api/v1/trends/trends-for-location/api-reference/get-trends-place

In [3]:
countries = {}
file = open("input/WOEID.txt")
countries = fd.dictionary(file, countries)

bearer_token = api.key['bearer_token']

search_url = "https://api.twitter.com/1.1/trends/place.json"

query_params = {'id': countries['UNITED_STATES']}

Está função usa o *bearer token* para poder realizar a autenticação, por mais que não irá logar no **twitter** o *bearer token* "representa uma autorização do Server emitida para o client. Por sua vez, o client deve possuir mecanismos próprios para identificar e validar o Token".

Fonte: [Bruno Brito](https://www.brunobrito.net.br/jwt-cookies-oauth-bearer/).

In [4]:
def bearer_oauth(r):
  '''
  Autenticação usando o bearer token
  '''
  r.headers["Authorization"] = f"Bearer {bearer_token}"
  r.headers["User-Agent"] = "v2RecentSearchPython"

  return r

Usando a url e o parametros necessario, a função abaixo retorna um *response* no formato **json**. Veja onde a função `bearer_oauth` será usada.

Fonte: https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/main/Recent-Search/recent_search.py

In [5]:
def connect_to_endpoint(url, params):
  '''
  Se conecta a url dada com o parametros dado.
  Consultar a documentação para saber o parametros nessario na pesquisa
  '''
  response = requests.get(url, auth=bearer_oauth, params=params)
  print(response.status_code)
  if response.status_code != 200:
    raise Exception(response.status_code, response.text)
        
  return response.json()

## Funcionalidade 2: Persistir os dados

1. criar conexão com o banco, neste projeto usando o MongoDb
2. Criar um banco chamado `tweets_db`
3. Criar uma coleção chamada `top_trends`
4. Inserir a *response* nesta coleção

Fonte: https://pymongo.readthedocs.io/en/stable/tutorial.html

### Variaveis de Ambiente

In [6]:
username = mongoEnv.key_mongo['mongo_username']
pwd = mongoEnv.key_mongo['mongo_password']

### Conexão com banco de dados

In [7]:
mongo_client = mongoDb.connect_to_mongo(username, pwd, '27017')

### Criação do Banco de Dados e a Coleção

In [8]:
database = mongoDb.create_db(mongo_client, 'tweets_db')
collection = database['top_trends']

### Conectando ao Endpoint da Twitter API

In [9]:
top_trends = connect_to_endpoint(search_url, query_params)

200


### Persistindo os dados encontrado no Json
Aqui eu decidi fazer um for para inserir um de cada vez, porém eu observei que só foi criado um unico documento, sendo que os top trendings é uma lista com os 50 top trendings da data atual.

In [10]:
for trend in top_trends:
    trend_id = collection.insert_one(trend).inserted_id

### Procurando e Printando o primeiro documento

In [11]:
pprint.pprint(collection.find_one())

{'_id': ObjectId('6146357d5b64b034788268e7'),
 'as_of': '2021-09-18T18:52:45Z',
 'created_at': '2021-09-17T10:31:04Z',
 'locations': [{'name': 'Brazil', 'woeid': 23424768}],
 'trends': [{'name': 'O DIABO VEM HOJE',
             'promoted_content': None,
             'query': '%22O+DIABO+VEM+HOJE%22',
             'tweet_volume': None,
             'url': 'http://twitter.com/search?q=%22O+DIABO+VEM+HOJE%22'},
            {'name': '#freebts',
             'promoted_content': None,
             'query': '%23freebts',
             'tweet_volume': None,
             'url': 'http://twitter.com/search?q=%23freebts'},
            {'name': 'JULIETTE E CASANEM',
             'promoted_content': None,
             'query': '%22JULIETTE+E+CASANEM%22',
             'tweet_volume': 12781,
             'url': 'http://twitter.com/search?q=%22JULIETTE+E+CASANEM%22'},
            {'name': 'ASSÉDIO NA RECORD',
             'promoted_content': None,
             'query': '%22ASS%C3%89DIO+NA+RECORD%22',
  

            {'name': 'NÃO É NÃO',
             'promoted_content': None,
             'query': '%22N%C3%83O+%C3%89+N%C3%83O%22',
             'tweet_volume': 1377762,
             'url': 'http://twitter.com/search?q=%22N%C3%83O+%C3%89+N%C3%83O%22'},
            {'name': 'MORE PROMOTIONS FOR LISA',
             'promoted_content': None,
             'query': '%22MORE+PROMOTIONS+FOR+LISA%22',
             'tweet_volume': 516742,
             'url': 'http://twitter.com/search?q=%22MORE+PROMOTIONS+FOR+LISA%22'}]}


### Printando o nome da coleção criada no banco de dados

In [12]:
database.list_collection_names()

['top_trends']